In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

num_data = 150

def sigmoid(x):
    return 1./(1+np.exp(-x))

df = pd.DataFrame({
    "x1": np.random.normal(size=num_data),
    "x2": np.random.normal(size=num_data),    
}).assign(
    prob_t = lambda df: sigmoid(df.x1+df.x2),
    t = lambda df: (df.prob_t >= np.random.uniform(size=num_data))*1,
    reward = lambda df: df.x1 - df.x2 + df.t * 2
)

df

,x1,x2,prob_t,t,reward
0,-0.677805,-1.030717,0.153356,0,0.352911
1,-2.692110,1.579317,0.247351,0,-4.271427
2,-0.335904,0.166042,0.457636,1,1.498053
3,-1.745885,-1.563609,0.035247,0,-0.182276
4,-0.419593,0.536323,0.529149,1,1.044084
...,...,...,...,...,...
145,-0.043139,0.256720,0.553193,1,1.700141
146,1.786006,0.418644,0.900666,1,3.367362
147,0.045460,0.188242,0.558161,0,-0.142783
148,0.564768,-1.228204,0.339968,1,3.792973


# IPS / DM

In [7]:
# IPS
mode = LogisticRegression().fit(X=df[["x1", "x2"]], y=df["t"]==1)
df["t_pred"] = mode.predict_proba(df[["x1", "x2"]])[:,1]

df["reward_treatment"] = df["reward"]*df["t"]/df["t_pred"]
df["reward_no_treatment"] = df["reward"]*(1-df["t"])/(1-df["t_pred"])

print((df["reward_treatment"]-df["reward_no_treatment"]).mean())

# DM
mode = LinearRegression().fit(X=df[["x1", "x2", "t"]], y=df["reward"])
df["reward_treatment"] = mode.predict(df[["x1", "x2", "t"]].assign(t=1))
df["reward_no_treatment"] = mode.predict(df[["x1", "x2", "t"]].assign(t=0))

print((df["reward_treatment"]-df["reward_no_treatment"]).mean())

1.830871230517721
2.0
